In [14]:
import tweepy
import pandas as pd
import os
import sys
import pymongo
import requests
import googletrans

ModuleNotFoundError: No module named 'googletrans'

In [13]:
import sys
sys.path.append("/usr/local/lib/python3.9/site_packages/googletrans") # go to parent dir

In [12]:
import sys
print(sys.path)

['/Users/iiyakhruschev/Desktop/_j/stream/src/python/ipynb', '/Users/iiyakhruschev/opt/anaconda3/lib/python37.zip', '/Users/iiyakhruschev/opt/anaconda3/lib/python3.7', '/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/lib-dynload', '', '/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages', '/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/iiyakhruschev/.ipython', '../', '/usr/local/lib/python3.9', '/usr/local/lib/python3.9/site_packages', '/usr/local/lib/python3.9/site_packages']


## Set Variables

In [39]:
consumer_key = os.getenv("CONSUMER_KEY")
consumer_secret = os.getenv("CONSUMER_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")

In [3]:
consumer_key ='49gyl5DGXciwxZqxEErGq2RXI'
consumer_secret ='J5Xgo2KqIRtIrzjgtcwkugK0gXXgrc1oACyW8mR6MgioQ0nZFk'
access_token = '1320711922581463043-gmuDE2eB3uev34ehJ17qooEs7L4JFn'
access_token_secret = 'Q1pwVXHEfE3RN9kKZmtDM9mbWM1QXoUzkUV99Z9GlOKq4'
bearer_token ='AAAAAAAAAAAAAAAAAAAAAKaMQgEAAAAAYE4alrVHLC%2B4tojodv%2B94fRfJOY%3DaqJfGxFeGTy4mINVhqFjCL2gPZMWOPcQXewKvWnCrqTroMsdBQ'

In [4]:
from pymongo import MongoClient
client_db =  'stream'
client_col = 'frenchgp' #args.collection

# source and target connections
client = MongoClient('mongodb://localhost:27017')

# batch size
batch_size = 1000

# database and collection names
db = client[client_db]
col = db[client_col]

## Build Stream

In [141]:
import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def auth():
#     return os.environ.get("BEARER_TOKEN")
    return bearer_token


def create_url():
#     return "https://api.twitter.com/2/tweets/sample/stream?tweet.fields=created_at&expansions=author_id&user.fields=created_at"
    return "https://api.twitter.com/2/tweets/search/stream"


def create_params():
    params = {'tweet.fields': 'created_at',
            'tweet.fields': 'public_metrics',
            'expansions': 'author_id,referenced_tweets.id,geo.place_id',
            'user.fields': 'created_at,location'}
    return params


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers, params=create_params(), stream=True)
    print(response.status_code)
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent=4, sort_keys=True))
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    timeout = 0
    while True:
        connect_to_endpoint(url, headers)
        timeout += 1


if __name__ == "__main__":
    main()

429
{
    "connection_issue": "TooManyConnections",
    "detail": "This stream is currently at the maximum allowed connection limit.",
    "title": "ConnectionException",
    "type": "https://api.twitter.com/2/problems/streaming-connection"
}


RuntimeError: The content for this response was already consumed

## Build Filter Stream

In [136]:

import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

headers = create_headers(bearer_token)

def get_stream(headers, set, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            col.insert_many(json.dumps(json_response, indent=4, sort_keys=True))
            print(json.dumps(json_response, indent=4, sort_keys=True))


def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)


if __name__ == "__main__":
    main()

Exception: Cannot get rules (HTTP 401): {"title":"Unauthorized","type":"about:blank","status":401,"detail":"Unauthorized"}

## MY WAY

In [39]:
rules = [{"value": "(formula1 OR #f1 OR 'formula one' OR 'formula 1') -indycar \
                    -'indy car' -is:retweet -is:reply -is:quote",
          "value": '-indycar -"indy car" -is:nullcast -is:retweeted -is:replied_to \
                    -is:quoted ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc \
                    OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR \
                    stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell \
                    OR mazepin OR latifi) (f1 OR formula1 OR "french grand prix" OR ricard OR \
                    "french GP" OR "grand prix" OR "formula 1" OR frenchgp)) -indycar \
                    -"indy car" -is:nullcast -is:retweet -is:reply -is:quote'}]
params = {
    'tweet.fields': 'created_at,public_metrics',
    'expansions': 'author_id,referenced_tweets.id,geo.place_id',
    'user.fields': 'created_at,location'
}

In [38]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def getRules():
    response2 = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=create_headers(bearer_token))
    print("GETTING RULES: ", response2.text)
    return response2.json()


def deleteRules():
    try:
        response2 = getRules()
        ids = [i['id'] for i in response2['data']]
        payload = {"delete": {"ids": ids}}
        response = requests.post(
            "https://api.twitter.com/2/tweets/search/stream/rules",
            headers=headers,
            json=payload)
        print("DELETING RULES")
    except:
#         print("NOT DELETED: ", response.text)
        pass


def setRules():
    payload = {"add": rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=create_headers(bearer_token),
        json=payload,
    )
    print("SET RULES: ", response.text)


def getStream(headers):
    with requests.get("https://api.twitter.com/2/tweets/search/stream",
                      headers=headers,
                      params=params,
                      stream=True) as response:
        print(response)
        for response_line in response.iter_lines():
            if response_line:
                json_response = json.loads(response_line)
                print(json.dumps(json_response, indent=4, sort_keys=True))
                col.insert(json.loads(response_line))
                
            

def main():
    headers = create_headers(bearer_token)
    deleteRules()
    setRules()
    getRules()
    while True:
        getStream(headers)
        timeout += 1


if __name__ == "__main__":
    main()

GETTING RULES:  {"data":[{"id":"1405185357482762243","value":" -is:retweet -is:reply -is:quote ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR \"french GP\" OR \"grand prix\" OR \"formula 1\" OR frenchgp))"}],"meta":{"sent":"2021-06-16T15:28:19.317Z"}}
DELETING RULES
SET RULES:  {"data":[{"value":" -is:retweet -is:reply -is:quote ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR \"french GP\" OR \"grand prix\" OR \"formula 1\" OR frenchgp))","id":"1405185482108157953"}],"meta":{"sent":"2021-06-16T15:28:22.115Z","summary":{"creat

/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{
    "data": {
        "author_id": "1350309702",
        "created_at": "2021-06-16T15:28:43.000Z",
        "geo": {},
        "id": "1405185576085733380",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 1,
            "retweet_count": 0
        },
        "text": "BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YEAR DEAL FOR ALPINE F1\nWHERE HE'S BEEN SINCE THE START OF 2020\nThis Following his debut at 2016 Belgian GP, 2 yrs at Force India, (Aston Martin), in 17 &amp; 18 with a year out in 19 as Mercedes 3rd Driver \nhas he earned that New contract?"
    },
    "includes": {
        "users": [
            {
                "created_at": "2013-04-13T21:55:37.000Z",
                "id": "1350309702",
                "location": "Stoke Ash, England",
                "name": "Steven Featley",
                "username": "SFeatley"
            }
        ]
    },
    "matching_rules": [
        {
            "id": 140518548210

KeyboardInterrupt: 

In [20]:
headers = create_headers(bearer_token)
r = requests.get("https://api.twitter.com/2/tweets/search/stream/rules",
                      headers=headers,
                      params=params,
                      stream=True)

In [34]:
rules = [{#"value": "(formula1 OR #f1) -indycar -'indy car' -is:nullcast -is:retweet -is:reply -is:quote",
          "value": ' -is:retweet -is:reply -is:quote ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR "french grand prix" OR ricard OR "french GP" OR "grand prix" OR "formula 1" OR frenchgp))'}]
params = {
    'tweet.fields': 'created_at,public_metrics',
    'expansions': 'author_id,referenced_tweets.id,geo.place_id',
    'user.fields': 'created_at,location'
}

In [35]:
payload = {"add": rules}
response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=create_headers(bearer_token),
        json=payload,
    )
response.text

'{"data":[{"value":" -is:retweet -is:reply -is:quote ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \\"french grand prix\\" OR ricard OR \\"french GP\\" OR \\"grand prix\\" OR \\"formula 1\\" OR frenchgp))","id":"1405183563151073280"}],"meta":{"sent":"2021-06-16T15:20:44.132Z","summary":{"created":1,"not_created":0,"valid":1,"invalid":0}}}'

In [36]:
getRules()

GETTING RULES:  {"data":[{"id":"1404970790316130308","value":"formula1 OR #f1"},{"id":"1405183354400645122","value":"((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR \"french GP\" OR \"grand prix\" OR \"formula 1\" OR frenchgp))"},{"id":"1405183563151073280","value":" -is:retweet -is:reply -is:quote ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR \"french GP\" OR \"grand prix\" OR \"formula 1\" OR frenchgp))"}],"meta":{"sent":"2021-06-16T15:21:00.540Z"}}


{'data': [{'id': '1404970790316130308', 'value': 'formula1 OR #f1'},
  {'id': '1405183354400645122',
   'value': '((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR "french grand prix" OR ricard OR "french GP" OR "grand prix" OR "formula 1" OR frenchgp))'},
  {'id': '1405183563151073280',
   'value': ' -is:retweet -is:reply -is:quote ((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR "french grand prix" OR ricard OR "french GP" OR "grand prix" OR "formula 1" OR frenchgp))'}],
 'meta': {'sent': '2021-06-16T15:21:00.540Z'}}

## Get Historic

In [53]:
  
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    tweet_fields = "tweet.fields=lang,author_id"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=1278747501642657792,1255542774432063488"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()

401


Exception: Request returned an error: 401 {"title":"Unauthorized","type":"about:blank","status":401,"detail":"Unauthorized"}

## Get User

In [54]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def auth():
    return bearer_token


def create_url():
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames=ik6292" #TwitterDev,TwitterAPI"
    user_fields = "user.fields=description,created_at"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by"
    return url

def create_params():
    params = {"usernames": "ik6292",
              "user.fields": "description,created_at,public_metrics"}
    return params

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, params=create_params(), headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()

200
{
    "data": [
        {
            "created_at": "2020-10-26T13:00:43.000Z",
            "description": "",
            "id": "1320711922581463043",
            "name": "ik629",
            "public_metrics": {
                "followers_count": 0,
                "following_count": 35,
                "listed_count": 0,
                "tweet_count": 3
            },
            "username": "ik6292"
        }
    ]
}
